Section 1 needs to be run to get the data.

Section 2 must be run to load the saved model and get sigma.

Section 3 can be run to get the hold-out likelihoods.

Section 4 is for creating data.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import time
from pickle import load
import tensorflow as tf
from tensorflow import keras
if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")

tf.random.set_seed(42)
K = keras.backend

from sklearn.preprocessing import StandardScaler,PowerTransformer
import math
from scipy.stats import multivariate_normal
from scipy.stats import norm
import os
import pandas as pd
import pickle
from helper import *


No GPU was detected. LSTMs and CNNs can be very slow without a GPU.


Run the below cell if using a GPU.

In [ ]:
#using laptop gpu
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [2]:
k = 8
J = 32
save_time_step = 0.005
h=1 
c=10
b=10

# 1: Prepare data #

In [3]:
#load train, validation and test datasets 
valid_dataset = np.load("../../../data/truth_run/climate_change_exp/val.npy")
train1 = np.load("../../../data/truth_run/climate_change_exp/train_set_1.npy")
train2 = np.load("../../../data/truth_run/climate_change_exp/train_set_2.npy")
train3 = np.load("../../../data/truth_run/climate_change_exp/train_set_3.npy")
f20 = np.load("../../../data/truth_run/training_dataset.npy")
train4 = np.concatenate([f20, 20*np.ones(shape=(f20.shape[0],k,1))],axis=2)[:200000]


test_set =  np.load("../../../data/truth_run/climate_change_exp/full_test_set.npy")
f_array = 23*np.ones((test_set.shape[0],k,1))
test_dataset = np.concatenate([test_set,f_array],axis=2)

test_dataset_20_w   = np.load("../../../data/truth_run/climate_eval_dataset.npy")
test_dataset_20 = np.concatenate([test_dataset_20_w,20*np.ones((test_dataset_20_w.shape[0],k,1))],axis=2)


In [4]:
def _rhs_X_dt(X, F,U,dt=0.005):
    """Compute the right hand side of the X-ODE."""

    dXdt = (-np.roll(X, 1,axis=1) * (np.roll(X, 2,axis=1) - np.roll(X, -1,axis=1)) -
                X + F - U)

    return dt * dXdt 


def U(Xt,Xt_1,F,dt=0.005):
    k1_X = _rhs_X_dt(Xt,F,U=0)
    k2_X = _rhs_X_dt(Xt + k1_X / 2,F, U=0)
    Xt_1_pred = k2_X + Xt 
    #print(Xt_1_pred)
    Ut = (Xt_1_pred - Xt_1 )/dt

    return Ut


def create_dataset_with_u(climate_eval_dataset,dt=0.005):
    """Shape of climate_eval_dataset is num_steps x k x 2"""
    
    data = climate_eval_dataset[:,:,0] #so we get num_steps x k shape    
    u_data =  U(data[:-1,:],data[1:,:],climate_eval_dataset[0,0,-1])    
    x_prev = data[:-1,:]
    
    return np.stack([u_data,x_prev],axis=2)
    
    

In [5]:
valid_dataset_new = create_dataset_with_u(valid_dataset)
test_dataset_new = create_dataset_with_u(test_dataset)
train1_new = create_dataset_with_u(train1)
train2_new = create_dataset_with_u(train2)
train3_new = create_dataset_with_u(train3)
train4_new = create_dataset_with_u(train4)

test_dataset_20_new = create_dataset_with_u(test_dataset_20)

In [6]:
history_length = 700

val_seq_length = 5000

In [7]:
def prepare_datasets_for_RNN(dataset,history_length):
    dataset = dataset[:(dataset.shape[0]//history_length)*history_length,:] # to make it multiple of history
    dataset_shape = dataset.shape[0]
    reshaped = dataset.reshape(int(dataset_shape/history_length),history_length,k,2)
    add_on = reshaped[1:,0,:,:]
    add_on = add_on.reshape(int(dataset_shape/history_length)-1,1,k,2)
    reshaped = reshaped[:-1,:,:,:]
    concat = np.concatenate((reshaped,add_on),axis=1)
    concat = concat.reshape(((int(dataset_shape/history_length)-1)*(history_length+1),k,2))
    concat = concat.reshape((k*(int(dataset_shape/history_length)-1)*(history_length+1),2),order="F")
    features = concat.reshape(k*(int(dataset_shape/history_length)-1),(history_length+1),2)
    
    return features

def prepare_train_dataset(train1,train2,train3,train4,history_length):
    """each of these train sets has a different F value """
    train_1_features = prepare_datasets_for_RNN(train1,history_length)
    train_2_features = prepare_datasets_for_RNN(train2,history_length)
    train_3_features = prepare_datasets_for_RNN(train3,history_length)
    train_4_features = prepare_datasets_for_RNN(train4,history_length)
    return np.concatenate([train_1_features,train_2_features,train_3_features,train_4_features],axis=0)

def prep_holdout(test_dataset):
    return np.transpose(test_dataset,[1,0,2])
    

In [8]:
train_nn_features = prepare_train_dataset(train1_new,train2_new,train3_new,train4_new,history_length)

valid_dataset_for_loglik = prepare_datasets_for_RNN(valid_dataset_new,val_seq_length)

test_nn_features = prep_holdout(test_dataset_new)

test_nn_features_20 = prep_holdout(test_dataset_20_new)

In [9]:
u_mean = train_nn_features[:,:,0].mean()
u_std = train_nn_features[:,:,0].std()

x_mean = train_nn_features[:,:,1].mean()
x_std = train_nn_features[:,:,1].std()

def scaled_data(input_dataset):
    data = input_dataset.copy()
    data[:,:,0] = (data[:,:,0] - u_mean)/u_std 
    data[:,:,1] = (data[:,:,1] - x_mean)/x_std 
    return data

train_nn_features = scaled_data(train_nn_features)
test_nn_features = scaled_data(test_nn_features)
valid_dataset_for_loglik = scaled_data(valid_dataset_for_loglik)
test_nn_features_20 = scaled_data(test_nn_features_20)


train_nn_input = train_nn_features[:,:-1,:]
train_nn_output = train_nn_features[:,1:,:]

test_nn_input = test_nn_features[:,:-1,:]
test_nn_output = test_nn_features[:,1:,:]

test_nn_20_input = test_nn_features_20[:,:-1,:]
test_nn_20_output =test_nn_features_20[:,1:,:]

valid_loglik_input = valid_dataset_for_loglik[:,:-1,:]
valid_loglik_output = valid_dataset_for_loglik[:,1:,:]


In [10]:
################################################################################################################

#Note that for loglik we multiply dt (0.005) by u_std as we are training a model in a scaled space. This factor 
#can be shown to arise if one considers a change of variables for likelihood as in the Appendix of our paper.

################################################################################################################


def loglik_gaussian_x(array,mean,sigma):
    sigma = sigma* tf.ones((mean.shape[0],mean.shape[1],1))
    term = -K.log((sigma**2) *2*math.pi) - tf.math.divide((array-mean),sigma)**2
    loglik = 0.5 * term 
    return loglik - np.log(u_std*0.005)

def calculate_sigma_mle(input_list,simulator):
    
    u_det = u_det_model.predict(input_list[0][:,:,-1:])
    u_true = input_list[0][:,:,:1]
    r = u_true - (u_det-u_mean)/u_std

    x_mean_out = simulator.predict([r,input_list[2],input_list[3]])[0]

    u_det_target = u_det_model.predict(input_list[1][:,:,-1:])
    u_true_target = input_list[1][:,:,:1]
    r_target = u_true_target - (u_det_target-u_mean)/u_std

    sigma_mle_x = (np.mean((x_mean_out- r_target)**2))**0.5

    return sigma_mle_x
    

def valid_loss(valid_list,simulator,sigma_x):
    """negative loglik for full generative model on validation set"""
    
    u_det = u_det_model.predict(valid_list[0][:,:,-1:])
    u_true = valid_list[0][:,:,:1]
    r = u_true - (u_det-u_mean)/u_std

    x_mean_out = simulator.predict([r,valid_list[2],valid_list[3]])[0]

    u_det_target = u_det_model.predict(valid_list[1][:,:,-1:])
    u_true_target = valid_list[1][:,:,:1]
    r_target = u_true_target - (u_det_target-u_mean)/u_std    
    
    loglik_x = loglik_gaussian_x(array=r_target,mean=x_mean_out,sigma=sigma_x)
    loss = -loglik_x
    return tf.reduce_mean(loss)

def loglik_gaussian(array,mean,sigma):
    sigma = sigma* tf.ones((mean.shape[0],mean.shape[1],1))
    term = -K.log((sigma**2) *2*math.pi) - tf.math.divide((array-mean),sigma)**2
    loglik = 0.5 * term 
    loglik = tf.reduce_mean(loglik)
    return loglik - np.log(u_std*0.005)

In [11]:
####### The below hidden states are the l part of the full (r,l) hidden state ###############

hidden_state_size = 4


hidden_in_train = np.zeros(shape=(train_nn_features.shape[0],hidden_state_size))
hidden_in_test = np.zeros(shape=(test_nn_features.shape[0],hidden_state_size))

hidden_in_test_20 = np.zeros(shape=(test_nn_features_20.shape[0],hidden_state_size))

hidden_in_valid_loglik = np.zeros(shape=(valid_dataset_for_loglik.shape[0],hidden_state_size))


input_list = [train_nn_input,train_nn_output,hidden_in_train,hidden_in_train]


# 2 : Load saved model #

In [12]:
rnn = keras.models.load_model("rnn_update.h5")
u_det_model = keras.models.load_model("u_nn.h5")

In [21]:
sigma_mle_x = calculate_sigma_mle(input_list,simulator=rnn)
sigma_mle_x

0.01575225355520782

# 3: Hold-out log-likelihood #

The loss is the negative log-lik so we just take the negative loss.

## F = 20 ##

In [32]:
test_list_20 = [test_nn_20_input,test_nn_20_output,hidden_in_test_20,hidden_in_test_20]

In [33]:
#10000 MTU
-valid_loss(test_list_20,rnn,0.015752254)

<tf.Tensor: shape=(), dtype=float32, numpy=6.530379>

## F = 23 ##

In [35]:
test_list = [test_nn_input,test_nn_output,hidden_in_test,hidden_in_test]

In [36]:
#10000 MTU
-valid_loss(test_list,rnn,0.015752254)

<tf.Tensor: shape=(), dtype=float32, numpy=5.8889995>

# 4: Generate data #

In [13]:
def simulate_rnn(initX,num_steps,F=20,rnn=rnn,u_det_model=u_det_model,dt=0.005,sigma=0.0157523):
    
    "num steps is #MTU/0.005"

    
    def _rhs_X_dt(X, U,dt=0.005):
        """Compute the right hand side of the X-ODE."""

        dXdt = (-np.roll(X, 1) * (np.roll(X, 2) - np.roll(X, -1)) -
                    X + F - U)
            
        return dt * dXdt 
    
    def sub_grid_u_term(X,array_r,hidden_state1,hidden_state2):
        U_det = u_det_model(((X-x_mean)/x_std).reshape(-1,1,1))
        #rnn
        feature = array_r
        pred_mean,hidden_out1,hidden_out2 = rnn([feature,hidden_state1,hidden_state2])
        pred_mean = pred_mean[:,0,:]
        noise = np.random.normal(size=(k,1))
        sample_r = pred_mean + noise*sigma
        u_scaled = (U_det[:,:,0] - u_mean)/u_std + sample_r
        u_unscaled = u_scaled*u_std + u_mean
        return u_unscaled[:,0],hidden_out1,hidden_out2,sample_r.numpy()
    
    
    def step(X_in,array_r,hidden_state1,hidden_state2):
        """Integrate one time step"""
        X = X_in.copy()
        k1_X = _rhs_X_dt(X, U=0)
        k2_X = _rhs_X_dt(X + k1_X / 2, U=0)

        U,h1,h2,r = sub_grid_u_term(X,array_r,hidden_state1,hidden_state2)
        
        X += k2_X
        
        X += - U * dt
        return X,h1,h2,r
            
    array = np.zeros(shape=(k,num_steps))
    array[:,0] = initX
    array_r = np.zeros(shape=(k,1,1))
    #array_r = np.random.normal(size=(k,1,1))
    hidden_state1 = np.zeros(shape=(k,hidden_state_size))
    hidden_state2 = np.zeros(shape=(k,hidden_state_size))
    
    for i in range(num_steps-1):
        X_new, h1,h2,r = step(array[:,i],array_r,hidden_state1,hidden_state2)
        array[:,i+1] = X_new
        array_r = r.reshape(k,1,1)
        hidden_state1 = h1
        hidden_state2 = h2
        
    return array

# can output last hidden state too to feed into the next arrays

The cell below is an example of how data is generated. Code of the form in the F=20 and F=23 subheadings is used to generate the full length data.

In [26]:
#example of generating data 

x_climate = np.load("../../../data/truth_run/climate_eval_dataset.npy")

init_X_climate = x_climate[0,:,0]

num_steps_climate = int(10000/0.005) #10000 MTU of data

data = simulate_rnn(init_X_climate,num_steps_climate,F=20)

## F = 20 Climate ##

In [41]:

# x_climate = np.load("../../../data/truth_run/training_dataset.npy")

# init_X_climate = x_climate[0,:,0]

# num_steps_climate = int(5000/0.005)
# initX_1 = init_X_climate
# x_array_climate = simulate_rnn(initX_1,num_steps_climate)
# np.save("../../../data/simulation_runs/rnn/rnn_final/f_20_data/1.npy",x_array_climate)



# initX_2 = x_array_climate[:,-1]
# extra2 = simulate_rnn(initX_2,num_steps_climate)[:,1:]
# np.save("../../../data/simulation_runs/rnn/rnn_final/f_20_data/2.npy",extra2)


############# and so on ##################


## F = 23 ##

In [80]:
# test_dataset = np.load("../../../data/truth_run/climate_change_exp/full_test_set.npy")
# init_X_climate_23 = test_dataset[0,:,0]
# num_steps_climate = int(6000/0.005) 

# initX_1 = init_X_climate_23
# x_array_climate = simulate_rnn(initX_1,num_steps_climate,23)
# np.save("../../../data/simulation_runs/rnn/rnn_final/clim_change/1.npy",x_array_climate)


# initX_2 = x_array_climate[:,-1]
# extra2 = simulate_rnn(initX_2,num_steps_climate,23)[:,1:]
# np.save("../../../data/simulation_runs/rnn/rnn_final/clim_change/2.npy",extra2)

# ############ and so on ##################



## Weather ##

In [32]:
def simulate_rnn_for_weather(starting_condition,num_steps,num_ensemble_members=40, rnn=rnn,u_det_model=u_det_model,sigma=0.0157523,F=20,dt=0.005):

    """
    starting_cond has shape num_starting_condxk
    """
    
    def _rhs_X_dt(X, U):
        """Compute the right hand side of the X-ODE."""

        dXdt = (-np.roll(X, 1,axis=1) * (np.roll(X, 2,axis=1) - np.roll(X, -1,axis=1)) -
                    X + F - U)
            
        return dt * dXdt 
    
    
    def sub_grid_u_term(X,array_r,hidden_state1,hidden_state2):
        U_det = u_det_model(((X-x_mean)/x_std).reshape(-1,1))
        #rnn
        feature = array_r
        pred_mean,hidden_out1,hidden_out2 = rnn([feature,hidden_state1,hidden_state2])
        pred_mean = pred_mean[:,0,:]
        noise = np.random.normal(size=(int(length*k),1))
        sample_r = pred_mean + noise*sigma
        
        u_scaled = (U_det[:,:,0] - u_mean)/u_std + sample_r
        u_unscaled = u_scaled*u_std + u_mean
        return tf.reshape(u_unscaled,(-1,k)),hidden_out1,hidden_out2,sample_r.numpy()
           
    
    def step(X_in,array_r,hidden_state1,hidden_state2):
        """Integrate one time step"""
        X = X_in.copy()
        k1_X = _rhs_X_dt(X, U=0)
        k2_X = _rhs_X_dt(X + k1_X / 2, U=0)

        U,h1,h2,r = sub_grid_u_term(X,array_r,hidden_state1,hidden_state2) ######
        
        X += k2_X
        
        X += - U * dt
        return X,h1,h2,r
        
    initX = starting_condition.repeat(num_ensemble_members,axis=0)  
        
    #np.random.seed(42)
    
    length = initX.shape[0]
    
    array = np.zeros(shape=(length,k,num_steps))
    array[:,:,0] = initX
    
    array_r = np.random.normal(size=(int(k*length),1,1))
    #array_r = np.zeros(shape=(int(k*length),1,1))
    hidden_state1 = np.zeros(shape=(int(k*length),hidden_state_size))
    hidden_state2 = np.zeros(shape=(int(k*length),hidden_state_size))
        
    for i in range(num_steps-1):
        print(i)
        X_new, h1,h2,r = step(array[:,:,i],array_r,hidden_state1,hidden_state2)
        array[:,:,i+1] = X_new
        array_r = r.reshape(-1,1,1)
        hidden_state1 = h1
        hidden_state2 = h2
        
    return array.reshape(starting_condition.shape[0],num_ensemble_members,k,num_steps)

In [33]:
truth_weather = np.load("../../../data/truth_run/weather_experiments/weather_array_longer.npy")

truth_weather_init_conditions = truth_weather[:,0,:]

num_init_conds = truth_weather.shape[0]
length_simulations = truth_weather.shape[1]
num_ensemble_members = 40


In [135]:
array = simulate_rnn_for_weather(truth_weather_init_conditions,length_simulations)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [136]:
np.save("../../../data/simulation_runs/rnn/rnn_final/f_20_data/weather_array_long_700.npy",array)